## A simple model for demand and supply of publicly-provided services in a city

In [ ]:
from enum import Enum
import os.path
import json

import numpy as np
import pandas as pd
import geopandas as gpd
import geopy, geopy.distance
import shapely
from sklearn import gaussian_process

from matplotlib import pyplot as plt 
import seaborn as sns
plt.rcParams['figure.figsize']= (20,14)

In [ ]:
## TODO: find way to put this into some global settings
import os
import sys
nb_dir = os.path.dirname(os.getcwd())
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg, istat_kpi

In [ ]:
from src.models.city_items import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model
from src.models.core import ServiceUnit, ServiceValues, ServiceEvaluator, \
    MappedPositionsFrame, DemandFrame, KPICalculator
from src.models.factories import UnitFactory, SchoolFactory, LibraryFactory 
from src.models.process_tools import GridMaker, ValuesPlotter, JSONWriter
from src.models.city_items import get_random_pos

In [ ]:
quicktest = [ServiceUnit(ServiceType.Library, 'Duomo', ageDiffusionIn=None), 
        ServiceUnit(ServiceType.Library, 'Ripamonti', 
                    position=geopy.Point(45.43, 9.201), ageDiffusionIn=None)]
ServiceEvaluator(quicktest).evaluate_services_at(MappedPositionsFrame(get_random_pos(4)))
del quicktest

In [ ]:
## Load scuole
scuoleFile =  '../data/processed/Milano_datiScuole.csv'
schoolLoader = UnitFactory.createLoader(ServiceType.School, scuoleFile)

# Initialise with a default lengthscale of 0.5 km
schoolUnits = schoolLoader.load(meanRadius=0.5)
schoolEval = ServiceEvaluator(schoolUnits)

In [ ]:
## Load biblioteche
bibliotecheFile =  '../data/processed/Milano_biblioteche.csv'
bibliotecheLoader = UnitFactory.createLoader(ServiceType.Library, bibliotecheFile)

# Initialise with a default lengthscale of 0.5 km
libraryUnits = bibliotecheLoader.load(meanRadius=0.5)

In [ ]:
# call grid making to discretise service evaluation, this is an alternative to evaluating on the demand units
milanoGridMK = GridMaker({'quartieri':'../data/raw/Milano_specific/Milano_quartieri.geojson'}, gridStep=.4)
testEvaluator = ServiceEvaluator(schoolUnits[:200])
# call evaluation on internal points only
valuesGrid = testEvaluator.evaluate_services_at(milanoGridMK.grid)
plotterNew = ValuesPlotter(valuesGrid)
plotterNew.plot_locations()
plotterNew.plot_service_levels(ServiceType.School) 

In [ ]:
frame = DemandFrame.create_from_istat_cpa('Milano')
demandTest = DemandFrame(frame.head(500).copy(), False)
tt = KPICalculator(demandTest, schoolUnits[::10], 'Milano')
kk = tt.compute_kpi_for_istat_values()
tt.compute_kpi_for_localized_services()

In [ ]:
plotterDemand = ValuesPlotter(tt.weightedValues)
plotterDemand.plot_service_levels(ServiceType.School)
plotterDemandEval = ValuesPlotter(tt.serviceValues)
plotterDemandEval.plot_service_levels(ServiceType.School)

In [ ]:
plotterNew = ValuesPlotter(ServiceValues(ServiceEvaluator(schoolUnits[::10]).servicePositions))
plotterNew.plot_service_levels(ServiceType.School)
plotterNew.plot_locations()

In [ ]:
testCalc = KPICalculator(frame, schoolUnits[:10] + libraryUnits[:10], 'Milano')
testCalc.compute_kpi_for_istat_values()
testCalc.compute_kpi_for_localized_services()
yy = JSONWriter(testCalc)
yy.write_all_files_to_default_path()